# Model Development

-----------

## Install necessary libraries

``` pip install tensorflow keras matplotlib scikit-learn ```

------

## Organize image data

In [ ]:
import os
import shutil
import random

# Define paths
original_dataset_dir = "./imgs/original"
base_dir = "./imgs"

# Split data
split_ratios = {'train': 0.7, 'validation': 0.15, 'test': 0.15}
for jump_type in os.listdir(original_dataset_dir):
    images = os.listdir(os.path.join(original_dataset_dir, jump_type))
    random.shuffle(images)
    
    train_end = int(len(images) * split_ratios['train'])
    val_end = train_end + int(len(images) * split_ratios['validation'])
    
    for i, image in enumerate(images):
        if i < train_end:
            split = 'train'
        elif i < val_end:
            split = 'validation'
        else:
            split = 'test'
        src = os.path.join(original_dataset_dir, jump_type, image)
        dest = os.path.join(base_dir, split, jump_type, image)
        shutil.copy(src, dest)

-----

## Model Work

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Set up paths
train_dir = "./imgs/train"
val_dir = "./imgs/validation"
test_dir = "./imgs/test"

### Data preparation

In [ ]:
IMG_SIZE = 224  # Standard size for ResNet50 input
BATCH_SIZE = 32

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

### Load pre-trained model

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers (optional)
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)  # Add a dense layer
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # Final output layer

model = Model(inputs=base_model.input, outputs=predictions)

### Compile the model

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [ ]:
EPOCHS = 10
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

---------

## Check performance

In [ ]:
# Plot Training and Validation Accuracy
plt.figure(figsize=(12, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot Training and Validation Loss
plt.figure(figsize=(12, 5))
plt.plot(history.history['loss'], label='Training Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

--------

## Persist the model

In [ ]:
# Save the model using the TensorFlow SavedModel format
model.save('gym-clf-model')